In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Метод $k$ ближайших соседей (kNN)

В этой части работы вы:
- научитесь готовить данные к построению модели (предобработка, или препроцессинг данных);
- познакомитесь с методами ближайших соседей для задач классификации, реализованными в библиотеке scikit-learn ;
- научитесь оценивать качество модели с помощью отложенной выборки.

In [ ]:
df = pd.read_csv('/kaggle/input/churn-in-telecoms-dataset/bigml_59c28831336c6604c800002a.csv')
df.head()

In [ ]:
df.info()

Нам нужно научиться предсказывать, будет ли клиент уходить из компании или нет (target - churn). Задача категориальная, нам надо предсказать к какой группе отноится пользователь

In [ ]:
sns.countplot(df['churn'])

Большая часть пользователей - лояльные пользователи, следовательно распределение далеко от нормального

In [ ]:
sns.countplot(df['state'])

Обработать данные штатов будет очень сложно, так как их больше 50, и они категориальные. Поэтому дропнем их.
Аналогично дропнем номера телефонов, вряд ли они несут полезную инфу

In [ ]:
df = df.drop(['state', 'phone number'], axis='columns')

In [ ]:
sns.countplot(df["area code"])

Имеем всего три значения area code. Преобразуем их в нормальный вид(0, 1, 2). Аналогично преобразуем в числа булевые столбцы

In [ ]:
df['area code'] = df['area code'].map({408: 0, 415: 1, 510: 2})
df['voice mail plan'] = df['voice mail plan'].map( {"no": 0,"yes": 1} )
df['international plan'] = df['international plan'].map( {"no": 0,"yes": 1} )
df['churn'] = df['churn'].map( {False: 0, True: 1 })
df.head()

Займемся разделением данных от таргета

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(df.drop('churn', axis='columns'))
Y = df['churn']
X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size = 0.25, random_state=42)

Попробуем натренировать дефолтный KNeighborsClassifier

In [ ]:
kc = KNeighborsClassifier(n_neighbors=1)
kc.fit(X_train, y_train)
Y_res = kc.predict(X_valid)
Y_res

In [ ]:
accuracy_score(y_valid, Y_res)

Вполне неплохо для элементарного алгоритма. Точность 85%. Однако даже давая на все вопросы False, наша модель имела бы приблизительно такой же результат. Давайте проверим модель кросс-валидацией

In [ ]:
fold = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(kc, X, Y,scoring='accuracy', cv=fold)
print(scores, scores.mean())

Поэкспериментируем с количеством соседей:

In [ ]:
grid = GridSearchCV(kc, {'n_neighbors': np.arange(1, 15)}, scoring='accuracy', cv=fold)
grid.fit(X_train, y_train)
print(grid.best_params_)
print(grid.best_score_)
grid_res = pd.DataFrame(grid.cv_results_)
grid_res.head()

Как видим, классификатор с гиперпараметром {5 соседей} показал себя лучше всех

In [ ]:
grid_res.plot(x='param_n_neighbors', y='mean_test_score')

Поэкспериментируем с различными метриками:

К слову, метрика f1, согласно с [этим](https://scikit-learn.org/stable/modules/model_evaluation.html), подходит нам гораздо больше. Посмотрим на результат:

In [ ]:
scores = cross_val_score(kc, X, Y, scoring='f1', cv=fold)
print(scores, scores.mean())

Окей, кажется наша модель выглядит немного туповато на данной метрике, проверим другие:

In [ ]:
kc_w = KNeighborsClassifier(n_neighbors=5, weights = 'distance')
kc_w.fit(X_train, y_train)

In [ ]:
kc_w_grid = GridSearchCV(kc_w, {"p": np.linspace(1,10, 20)}, scoring="accuracy", cv = fold)
kc_w_grid.fit(X_train, y_train)

In [ ]:
print(kc_w_grid.best_params_)
print(kc_w_grid.best_score_)
grid_res = pd.DataFrame(kc_w_grid.cv_results_)
grid_res.head()

In [ ]:
grid_res.plot(x='param_p', y='mean_test_score')

Проверим, как RadiusNeighborsClassifier в плане наших данных:

In [ ]:
from sklearn.neighbors import RadiusNeighborsClassifier

cf_rad = RadiusNeighborsClassifier(radius=10)
cf_rad.fit(X_train, y_train)

scores = cross_val_score(cf_rad, X, Y, scoring='accuracy', cv=fold)
print(scores, scores.mean())

Не сильно лучше, а учитывая что данная метрика для нашей задачи не очень подходит, то и результатов мы особых не получили)